In [1]:
import pandas as pd
import numpy as np

In [2]:
# ! pip install pymysql

In [2]:
# mysql에서 db 불러오기

import pymysql
user_rating = pymysql.connect(
    user = 'root',
    passwd = '',
    host = 'localhost',
    db = 'itemdb',
    charset = 'utf8'
)


### CF 정확도 개선을 위한 신뢰도 가중 방법

In [3]:
rating_matrix = pd.read_csv('CollabDB.csv')
rating_matrix

FileNotFoundError: [Errno 2] No such file or directory: 'CollabDB.csv'

In [5]:
rm =rating_matrix.copy()

In [6]:
rating_matrix.set_index(keys='UID', inplace=True)

In [7]:
from sklearn.metrics.pairwise import cosine_similarity
matrix_dummy = rating_matrix.copy().fillna(0)
user_similarity = cosine_similarity(matrix_dummy, matrix_dummy)
user_similarity = pd.DataFrame(user_similarity, index=rating_matrix.index, columns=rating_matrix.index)
user_similarity 

UID,2023052701,2023052702,2023052703,2023052704,2023052705,2023052706,2023052707,2023052708,2023052709,2023052710,2023052711,2023052712,2023052713,2023052714,2023052715,2023052716,2023052717,2023052718,2023052719,2023052720
UID,,,,,,,,,,,,,,,,,,,,
2023052701,1.000000,0.733038,0.738995,0.735941,0.762387,0.741804,0.730896,0.755547,0.749769,0.754039,0.754998,0.749084,0.752360,0.741406,0.748805,0.752532,0.741792,0.756320,0.749026,0.765075
2023052702,0.733038,1.000000,0.745589,0.744875,0.738644,0.743252,0.737714,0.745536,0.741177,0.754970,0.760490,0.746398,0.750243,0.744782,0.753651,0.741996,0.744315,0.733209,0.745987,0.755238
2023052703,0.738995,0.745589,1.000000,0.744401,0.764161,0.758963,0.751733,0.765423,0.747167,0.745440,0.753261,0.748932,0.749107,0.732769,0.749393,0.749285,0.749101,0.742102,0.751831,0.737532
2023052704,0.735941,0.744875,0.744401,1.000000,0.750346,0.738938,0.737940,0.734064,0.738191,0.752317,0.763323,0.761290,0.756613,0.744487,0.736253,0.760188,0.771123,0.746414,0.755655,0.765558
2023052705,0.762387,0.738644,0.764161,0.750346,1.000000,0.748272,0.744188,0.763590,0.749795,0.764706,0.771416,0.756738,0.760085,0.736449,0.761082,0.740593,0.763927,0.752590,0.752890,0.755086
2023052706,0.741804,0.743252,0.758963,0.738938,0.748272,1.000000,0.724655,0.747899,0.719056,0.731094,0.746669,0.735391,0.737801,0.751910,0.738569,0.728421,0.745499,0.731601,0.746084,0.748115
2023052707,0.730896,0.737714,0.751733,0.737940,0.744188,0.724655,1.000000,0.727498,0.716311,0.745309,0.750415,0.735642,0.731369,0.738042,0.737033,0.748779,0.751157,0.744924,0.760415,0.734374
2023052708,0.755547,0.745536,0.765423,0.734064,0.763590,0.747899,0.727498,1.000000,0.736898,0.734942,0.757214,0.736007,0.737238,0.736271,0.746255,0.731425,0.749790,0.753462,0.742396,0.757445
2023052709,0.749769,0.741177,0.747167,0.738191,0.749795,0.719056,0.716311,0.736898,1.000000,0.742572,0.753367,0.732723,0.743745,0.746596,0.741725,0.731086,0.739104,0.720996,0.746040,0.739392


In [8]:
rating_mean = rating_matrix.mean(axis=1) # 열 기준 평균(제품에 대한 사용자들의 평균)
rating_bias = (rating_matrix.T - rating_mean).T
rating_bias

,0,1,2,3,4,5,6,7,8,9,...,796,797,798,799,800,801,802,803,804,805
UID,,,,,,,,,,,,,,,,,,,,,
2023052701,-1.741315,-1.741315,-0.741315,-1.741315,-0.241315,2.258685,1.758685,-1.241315,-2.241315,-0.241315,...,-0.741315,2.258685,-0.241315,-1.241315,1.258685,1.758685,0.758685,1.758685,-1.241315,1.758685
2023052702,1.242556,-2.757444,-2.757444,0.242556,-2.757444,-2.257444,2.242556,1.742556,-2.257444,1.742556,...,-2.757444,-0.257444,0.242556,-0.257444,1.242556,0.742556,0.742556,0.742556,0.242556,-1.257444
2023052703,1.743176,0.243176,0.243176,1.743176,0.243176,2.243176,-1.256824,-1.256824,1.743176,1.243176,...,-2.756824,-1.256824,0.743176,-2.756824,-1.756824,2.743176,0.243176,0.243176,-0.756824,-2.256824
2023052704,-1.843052,2.156948,0.656948,-1.843052,-1.343052,1.156948,-1.343052,2.156948,2.156948,-1.843052,...,-1.843052,-1.843052,1.156948,0.656948,0.156948,0.156948,-1.843052,1.656948,2.156948,1.656948
2023052705,0.684243,0.684243,0.184243,0.184243,-1.315757,1.684243,-0.315757,-0.815757,2.184243,-0.315757,...,0.684243,1.184243,1.684243,1.184243,-0.315757,-0.815757,0.184243,0.184243,-2.315757,1.684243
2023052706,-0.705335,-0.705335,-2.205335,2.294665,-2.205335,1.294665,1.794665,-1.705335,-0.205335,1.294665,...,0.794665,2.794665,-2.205335,-0.705335,-0.705335,2.294665,-1.705335,1.294665,-1.705335,-2.205335
2023052707,2.797146,2.297146,-0.202854,1.297146,-1.202854,-2.202854,2.297146,1.297146,0.797146,-2.702854,...,0.297146,-0.202854,-1.702854,2.797146,0.297146,-1.202854,0.297146,-1.202854,2.297146,0.297146
2023052708,1.276055,-1.223945,-1.223945,2.276055,-1.223945,-1.723945,-0.723945,1.776055,-0.723945,2.276055,...,2.776055,0.276055,-2.223945,-0.223945,-2.223945,-0.223945,-0.223945,2.776055,-0.223945,-2.223945
2023052709,2.816998,-2.183002,1.816998,0.816998,-0.183002,-2.683002,-0.183002,2.316998,0.816998,2.316998,...,2.316998,-2.183002,2.316998,1.316998,0.816998,-0.183002,0.816998,-1.683002,-2.183002,0.816998


In [9]:
# 사용자별 공통 평가 수 계산
rating_binary1 = np.array((rating_matrix > 0).astype(float))
rating_binary2 = rating_binary1.T
counts = np.dot(rating_binary1, rating_binary2)
counts = pd.DataFrame(counts, index=rating_matrix.index, columns=rating_matrix.index).fillna(0)
counts

UID,2023052701,2023052702,2023052703,2023052704,2023052705,2023052706,2023052707,2023052708,2023052709,2023052710,2023052711,2023052712,2023052713,2023052714,2023052715,2023052716,2023052717,2023052718,2023052719,2023052720
UID,,,,,,,,,,,,,,,,,,,,
2023052701,781.0,742.0,747.0,746.0,748.0,731.0,738.0,736.0,735.0,747.0,747.0,741.0,752.0,735.0,754.0,743.0,748.0,740.0,739.0,748.0
2023052702,742.0,765.0,732.0,731.0,733.0,715.0,722.0,722.0,721.0,731.0,733.0,727.0,735.0,718.0,739.0,728.0,730.0,722.0,726.0,731.0
2023052703,747.0,732.0,772.0,736.0,739.0,721.0,731.0,725.0,728.0,736.0,742.0,733.0,743.0,725.0,745.0,735.0,741.0,728.0,732.0,740.0
2023052704,746.0,731.0,736.0,770.0,737.0,722.0,728.0,724.0,725.0,738.0,740.0,730.0,739.0,725.0,745.0,731.0,737.0,727.0,731.0,736.0
2023052705,748.0,733.0,739.0,737.0,772.0,722.0,731.0,727.0,728.0,737.0,741.0,734.0,745.0,727.0,746.0,736.0,737.0,727.0,730.0,740.0
2023052706,731.0,715.0,721.0,722.0,722.0,754.0,714.0,710.0,711.0,720.0,722.0,719.0,724.0,710.0,728.0,715.0,722.0,711.0,712.0,722.0
2023052707,738.0,722.0,731.0,728.0,731.0,714.0,763.0,717.0,718.0,729.0,731.0,724.0,734.0,718.0,737.0,726.0,728.0,721.0,723.0,732.0
2023052708,736.0,722.0,725.0,724.0,727.0,710.0,717.0,759.0,713.0,725.0,726.0,721.0,729.0,711.0,733.0,722.0,729.0,717.0,719.0,725.0
2023052709,735.0,721.0,728.0,725.0,728.0,711.0,718.0,713.0,759.0,724.0,725.0,724.0,731.0,715.0,735.0,723.0,725.0,715.0,719.0,729.0


In [10]:
def CF_knn_bias_sig(user_id, item_id, neighbor_size=0):
    if str(item_id) in rating_bias.columns:
        # 현 user와 다른 사용자 간의 유사도 가져오기
        sim_scores = user_similarity[user_id]
        # 현 movie의 평점편차 가져오기 (여기 코드 바뀜)
        item_ratings = rating_bias.iloc[:,item_id]   # ((((((이건 item_id가 열로 들어가도록, 리스트 형태로 받아옴))))))
        # 현 item에 대한 rating이 없는 사용자 표시
        no_rating = item_ratings.isnull()
        # 현 사용자와 다른 사용자간 공통 평가 아이템 수 가져오기 
        common_counts = counts[user_id]
        # 공통으로 평가한 제품의 수가 SIG_LEVEL보다 낮은 사용자 표시
        low_significance = common_counts < SIG_LEVEL
        # 평가를 안 하였거나, SIG_LEVEL이 기준 이하인 user 제거
        none_rating_idx = item_ratings[no_rating | low_significance].index
        item_ratings = item_ratings.drop(none_rating_idx)
        sim_scores = sim_scores.drop(none_rating_idx)
##### (2) Neighbor size가 지정되지 않은 경우        
        if neighbor_size == 0:
            # 편차로 예측값(편차 예측값) 계산
            prediction = np.dot(sim_scores, item_ratings) / sim_scores.sum()
            # 편차 예측값에 현 사용자의 평균 더하기
            prediction = prediction + rating_mean[user_id]
##### (3) Neighbor size가 지정된 경우            
        else:
            # 해당 영화를 평가한 사용자가 최소 MIN_RATINGS 이상인 경우에만 계산            
            if len(sim_scores) > MIN_RATINGS:
                # 지정된 neighbor size 값과 해당 영화를 평가한 총사용자 수 중 작은 것으로 결정
                neighbor_size = min(neighbor_size, len(sim_scores))
                # array로 바꾸기 (argsort를 사용하기 위함)
                sim_scores = np.array(sim_scores)
                movie_ratings = np.array(item_ratings)
                # 유사도를 순서대로 정렬
                user_idx = np.argsort(sim_scores)
                # 유사도와 rating을 neighbor size만큼 받기
                sim_scores = sim_scores[user_idx][-neighbor_size:]
                item_ratings = item_ratings[user_idx][-neighbor_size:]
                # 편차로 예측치 계산
                prediction = np.dot(sim_scores, item_ratings) / sim_scores.sum()
                # 예측값에 현 사용자의 평균 더하기
                prediction = prediction + rating_mean[user_id]
            else:
                prediction = rating_mean[user_id]
    else:
        prediction = rating_mean[user_id]
    return prediction

SIG_LEVEL = counts.mean().mean()  #750
MIN_RATINGS = counts.min().mean()  # 공통 제품 개수의 임계치

In [11]:
recommend_list = pd.DataFrame(columns = ['UID', 'item', 'prediction'])
for i in rating_matrix.index:
    for j in range(len(rating_matrix.columns)):
        if rating_matrix.loc[i][j]==0 :
            pred = CF_knn_bias_sig(i, j, neighbor_size=0)
            print("UID:", i, "item:" , j, "prediction:", pred)
            recommend = pd.DataFrame({'UID':[i], 'item' : [j], 'prediction': [pred]})
            recommend_list = pd.concat([recommend_list, recommend], ignore_index=True)
recommend_list

UID: 2023052701 item: 34 prediction: 2.68038387680843
UID: 2023052701 item: 67 prediction: 1.666521975187387
UID: 2023052701 item: 84 prediction: 2.5725338486676237
UID: 2023052701 item: 99 prediction: 2.911941923630762
UID: 2023052701 item: 134 prediction: 2.6525737760817414
UID: 2023052701 item: 158 prediction: 2.4158706737552236
UID: 2023052701 item: 164 prediction: 2.4500657080925055
UID: 2023052701 item: 200 prediction: 2.906846968028862
UID: 2023052701 item: 240 prediction: 2.315969878215272
UID: 2023052701 item: 253 prediction: 2.4952034149822606
UID: 2023052701 item: 264 prediction: 2.7093558078366518
UID: 2023052701 item: 266 prediction: 2.3127671236121943
UID: 2023052701 item: 345 prediction: 3.2976096932128893
UID: 2023052701 item: 389 prediction: 2.4440074844449793
UID: 2023052701 item: 417 prediction: 2.91592098305802
UID: 2023052701 item: 427 prediction: 2.4368257013825483
UID: 2023052701 item: 434 prediction: 2.911522367549485
UID: 2023052701 item: 468 prediction: 2.7855

UID: 2023052705 item: 622 prediction: 2.710257077501598
UID: 2023052705 item: 628 prediction: 3.2686502571333924
UID: 2023052705 item: 653 prediction: 2.067817483972051
UID: 2023052705 item: 666 prediction: 2.740909422789956
UID: 2023052705 item: 675 prediction: 2.8992605837378655
UID: 2023052705 item: 696 prediction: 2.555016418413386
UID: 2023052705 item: 699 prediction: 2.6324720990890724
UID: 2023052705 item: 711 prediction: 2.6749968667052832
UID: 2023052705 item: 739 prediction: 2.6786824260531485
UID: 2023052705 item: 773 prediction: 3.268088332536949
UID: 2023052705 item: 779 prediction: 2.444356195176388
UID: 2023052705 item: 781 prediction: 2.753979164299341
UID: 2023052706 item: 12 prediction: 0.0
UID: 2023052706 item: 37 prediction: 0.0
UID: 2023052706 item: 41 prediction: 0.0
UID: 2023052706 item: 58 prediction: 0.0
UID: 2023052706 item: 73 prediction: 0.0
UID: 2023052706 item: 89 prediction: 0.0
UID: 2023052706 item: 93 prediction: 0.0
UID: 2023052706 item: 100 prediction

UID: 2023052711 item: 437 prediction: 1.6586420938011388
UID: 2023052711 item: 446 prediction: 2.151461140229473
UID: 2023052711 item: 461 prediction: 3.27696253038357
UID: 2023052711 item: 499 prediction: 2.5591799731940914
UID: 2023052711 item: 515 prediction: 2.334342964260613
UID: 2023052711 item: 541 prediction: 1.7295058673014119
UID: 2023052711 item: 575 prediction: 2.6323592118987196
UID: 2023052711 item: 645 prediction: 1.9610979969452256
UID: 2023052711 item: 664 prediction: 2.5180502418851005
UID: 2023052711 item: 678 prediction: 2.4044837224651303
UID: 2023052711 item: 686 prediction: 2.519322574862748
UID: 2023052711 item: 696 prediction: 2.5550388922161886
UID: 2023052711 item: 700 prediction: 2.7892349286022915
UID: 2023052711 item: 714 prediction: 2.782670358837214
UID: 2023052711 item: 738 prediction: 2.446410124549163
UID: 2023052712 item: 0 prediction: 2.173951104490349
UID: 2023052712 item: 3 prediction: 2.347019805589041
UID: 2023052712 item: 12 prediction: 2.49552

UID: 2023052719 item: 323 prediction: 0.8232530958621749
UID: 2023052719 item: 325 prediction: 2.2071494694556955
UID: 2023052719 item: 331 prediction: 1.2806091213959114
UID: 2023052719 item: 346 prediction: 2.203240676389677
UID: 2023052719 item: 365 prediction: 1.7436073757635346
UID: 2023052719 item: 367 prediction: 2.6678995681007
UID: 2023052719 item: 376 prediction: 2.6661952517026193
UID: 2023052719 item: 424 prediction: 2.0882668326511196
UID: 2023052719 item: 432 prediction: 1.9740115444558244
UID: 2023052719 item: 463 prediction: 2.435863881434798
UID: 2023052719 item: 481 prediction: 1.9762597001784432
UID: 2023052719 item: 508 prediction: 1.9744680256709994
UID: 2023052719 item: 515 prediction: 2.5507649267488866
UID: 2023052719 item: 525 prediction: 1.973365787337031
UID: 2023052719 item: 536 prediction: 1.1679562318044414
UID: 2023052719 item: 559 prediction: 2.782800613414789
UID: 2023052719 item: 578 prediction: 2.0888397913454444
UID: 2023052719 item: 608 prediction: 

,UID,item,prediction
0,2023052701,34,2.680384
1,2023052701,67,1.666522
2,2023052701,84,2.572534
3,2023052701,99,2.911942
4,2023052701,134,2.652574
...,...,...,...
766,2023052720,750,2.495837
767,2023052720,789,2.986496
768,2023052720,799,2.095799
769,2023052720,801,2.662986


In [12]:
# 하나의 값만 확인
CF_knn_bias_sig(2023052701, 0, neighbor_size=0)

2.7715868676959854

In [13]:
# 전체 사용자 df) UID별로 예상평점이 높은 제품들부터 보여주기
recommend_list = recommend_list.sort_values(by=['UID', 'prediction'], ascending=[True, False], ignore_index=True)
recommend_list

,UID,item,prediction
0,2023052701,687,3.525378
1,2023052701,751,3.402042
2,2023052701,345,3.297610
3,2023052701,530,3.220460
4,2023052701,500,2.924688
...,...,...,...
766,2023052720,749,2.055109
767,2023052720,315,2.049941
768,2023052720,179,1.836511
769,2023052720,588,1.770759


In [14]:
# 특정 UID에게 추천할 상품 순차목록
recommend_list[recommend_list['UID']==2023052701]

,UID,item,prediction
0,2023052701,687,3.525378
1,2023052701,751,3.402042
2,2023052701,345,3.297610
3,2023052701,530,3.220460
4,2023052701,500,2.924688
5,2023052701,417,2.915921
6,2023052701,99,2.911942
7,2023052701,434,2.911522
8,2023052701,200,2.906847
9,2023052701,468,2.785537


### 평점이 높다고 예상되는 제품 추천
메모리 기반 CF 알고리즘은 모든 사용자 간의 평점의 유사도를 계산하고 유사도에 따라 타겟 사용자가 평가하지 않은 모든 아이템에 대해 예상 평점을 구한다. 예상 평점은 다른 유사 사용자의 해당 아이템에 대한 평점을 타겟 사용자와의 유사도로 가중 평균해서 구한다. 아이템 중에서 가장 높은 평점의 N개의 아이템을 추천한다.

In [ ]:

### 성능 평가

# RMSE 함수
def RMSE(y_true, y_pred):
    return np.sqrt(np.mean((np.array(y_true)-np.array(y_pred))**2))

# score(RMSE) 계산
def score(model):
    id_pairs = zip(x_test['user_id'], x_test['movie_id'])
    y_pred = np.array([model(user, movie) for (user,movie) in id_pairs])
    y_true = np.array(x_test['rating'])
    return RMSE(y_true, y_pred)
    
### 데이터셋 만들기
x = ratings.copy()
y = ratings['user_id']

x_train, x_test, y_train, y_test = train_test_split(x, y, test_size = 0.25, stratify = y)

ratings_matrix = x_train.pivot(index='user_id', columns = 'movie_id',values = 'rating')